In [1]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

import numpy as np
import tensorflow as tf
import random
import time

class MNIST:
        
    def __init__(self):
        print("init")
        
    def one_hot(self, Y_int, max_num):
        return np.eye(max_num)[Y_int]
    
    def load_test_data(self, csv_file_path):
        x = np.loadtxt(csv_file_path, delimiter=',', dtype=np.int32)
        self.TestX = x/255.0               # data : [None, 784]
        
        
    def load_train_data(self, csv_file_path, divide_ratio, nb_classes):
        
        xy = np.loadtxt(csv_file_path, delimiter=',', dtype=np.int32)
        
        total_size = len(xy)
        divide_num = int( total_size*divide_ratio );
        
        self.TrainX = xy[:divide_num, 1:]/255.0               # data : [None, 784]
        self.TrainY_int = xy[:divide_num, [0]]                # label : [None, 1]
        self.TrainY = self.one_hot(self.TrainY_int, nb_classes)       # one hot : [None, 1, 10]
        self.TrainY = np.reshape(self.TrainY, [-1, nb_classes])     # one hot : [None, 10]
        
        # print(self.TestX.shape)
        self.ValX = xy[divide_num:, 1:]/255.0                # data : [None, 784]
        self.ValY_int = xy[divide_num:, [0]]                 # label : [None, 1]
        self.ValY = self.one_hot(self.ValY_int, nb_classes)         # one hot : [None, 1, 10]
        self.ValY = np.reshape(self.ValY, [-1, nb_classes])       # one hot : [None, 10]
        
    def next_batch(self, batch_size):
        # random int를 갯수만큼 수행하여 해당 index의 X와 Y를 리턴한다.
        sample_indices = np.random.randint(0, len(self.TrainY), size=batch_size)
        return self.TrainX[sample_indices, :], self.TrainY[sample_indices, :]
       
    def train(self, batch_size, traing_epoch):
        # Data Initialize
        X = tf.placeholder(tf.float32, shape=[None, 784])
        X_img = tf.reshape(X, [-1, 28, 28, 1]) # img 28*28*1(BW Image)
        Y = tf.placeholder(tf.float32, shape=[None, 10])
        keep_prob = tf.placeholder(tf.float32)

        # L1 ImgIn shape=(?, 28, 28, 1)
        W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
        # Conv -> (?, 28, 28, 32)
        L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
        L1 = tf.nn.relu(L1)
        # Pool -> (?, 14, 14, 32)
        L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], 
                            strides=[1, 2, 2, 1], padding='SAME')
        L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

        # L2 ImgIn shape=(?, 14, 14, 32)
        W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
        # Conv -> (?, 14, 14, 64)
        L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
        L2 = tf.nn.relu(L2)
        # Pool -> (?, 7, 7, 64)
        L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], 
                            strides=[1, 2, 2, 1], padding='SAME')
        L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

        # L3 ImgIn shape=(?, 7, 7, 128)
        W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
        # Conv -> (?, 7, 7, 128)
        L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
        L3 = tf.nn.relu(L3)
        # Pool -> (?, 4, 4, 128)
        L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], 
                            strides=[1, 2, 2, 1], padding='SAME')
        L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

        # reshape = (?, 3136)
        L3 = tf.reshape(L3, [-1, 4*4*128])

        # L4 fully Connected Layer
        W4 = tf.get_variable("W4", shape=[4*4*128, 625], initializer=tf.contrib.layers.xavier_initializer())
        b4 = tf.Variable(tf.random_normal([1, 625]))
        L4 = tf.nn.relu(tf.matmul(L3, W4)+b4)
        L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

        # L5 Final FC 625 input -> 10 output
        W5 = tf.get_variable("W5", shape=[625, 10], initializer=tf.contrib.layers.xavier_initializer())
        b5 = tf.Variable(tf.random_normal([1, 10]))

        # define Hypothesis
        logits = tf.matmul(L4, W5)+b5
        hypothesis = tf.nn.softmax(logits)

        # define cost
        cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
        cost = tf.reduce_mean(cost_i)

        # define gradient
        gradient = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

        # calc accuracy
        prediction = tf.argmax(hypothesis, 1)
        answer = tf.argmax(Y, 1)
        accuracy = tf.reduce_mean( tf.cast( tf.equal(prediction, answer), dtype=tf.float32)) 

        # init network
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())

        # go implement
        total_recur = 0

        start_time = time.time()
        for epoch in range(traing_epoch):
            avg_cost = 0.0
            total_batch = int(len(self.TrainY)/batch_size)

            for batch in range(total_batch):
                batch_X, batch_Y = self.next_batch(batch_size)
                cost_, _ = sess.run([cost, gradient], feed_dict={X: batch_X, Y:batch_Y, keep_prob:0.7})
                avg_cost += cost_/total_batch
                total_recur = total_recur + 1

            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

        end_time = time.time()
        print('Learning Time = ', end_time-start_time)
        # implement accuracy
        print("Accuracy: ", sess.run(accuracy, feed_dict={X: self.ValX, Y:self.ValY, keep_prob:1.0} ) )
        
        print(sess.run(prediction, feed_dict={X: self.ValX, keep_prob:1.0}))
        print(sess.run(answer, feed_dict={Y: self.ValY, keep_prob:1.0}))
        
        predict = sess.run(prediction, feed_dict={X: self.TestX, keep_prob:1.0})
        np.savetxt("./predict.csv", predict, delimiter=",")

        
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'1J7BI4Gvu1-1-Zgqpw6VgDmjVdmP0QS5Z'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv')

# downloaded = drive.CreateFile({'id':'1NNPxbxX65me8LbQblenrGvyFcEqOm946'}) # replace the id with id of file you want to access
# downloaded.GetContentFile('test_tiny.csv')

downloaded = drive.CreateFile({'id':'1qd3idtK9SvfKubj7f2FRGmdUMjXtBQI-'}) # replace the id with id of file you want to access
downloaded.GetContentFile('test.csv')
        
test = MNIST()
test.load_train_data('train.csv', 0.7, 10)
test.load_test_data('test.csv')
test.train(50, 50)

files.download('./predict.csv')

init
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch: 0001 cost = 0.448497932
Epoch: 0002 cost = 0.112512294
Epoch: 0003 cost = 0.089538135
Epoch: 0004 cost = 0.071975009
Epoch: 0005 cost = 0.057651816
Epoch: 0006 cost = 0.053585116
Epoch: 0007 cost = 0.048262265
Epoch: 0008 cost = 0.042739795
Epoch: 0009 cost = 0.039808381
Epoch: 0010 cost = 0.039720183
Epoch: 0011 cost = 0.033897462
Epoch: 0012 cost = 0.036678755
Epoch: 0013 cost = 0.033453652
Epoch: 0014 cost = 0.033176561
Epoch: 0015 cost = 0.029673411
Epoch: 0016 cost = 0.031965396
Epoch: 0017 cost = 0.027027064
Epoch: 0018 cost = 0.024619887
Epoch: 0019 cost = 0.024855465
Epoch: 0020 cost = 0.024299113
Epoch: 0021 cost = 0.026166432
Epoch: 0022 cost = 0.025793536
Epoch: 0023 cost = 0.022749806
Epoch: 0024 cost = 0.022777967
Epoch: 0025 cost = 0.020522322
Epoch: 0026 cost = 0.023228076


MessageError: ignored

In [0]:
files.download('./predict.csv')

![tensorboard](./김민욱_Score.JPG)